# Hierarchically Deep Convolutional Neural Network For Image Recognition

## Setup and Imports

**Import Packages**

In [ ]:
import keras as kr
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()


from keras.datasets import cifar100

from sklearn.model_selection import train_test_split

from random import randint
import time
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
if not os.path.exists('data/models/'):
    os.mkdir('data/models')

**Define Global Variables**

In [ ]:
# The number of coarse categories
coarse_categories = 9

# The number of fine categories
fine_categories = 25

## Import and Preprocess Dataset

In [ ]:
import random
import keras_preprocessing.image

def load_and_crop_img(path, grayscale=False, color_mode='rgb', target_size=None,
             interpolation='nearest'):
    """Wraps keras_preprocessing.image.utils.loag_img() and adds cropping.
    Cropping method enumarated in interpolation
    # Arguments
        path: Path to image file.
        color_mode: One of "grayscale", "rgb", "rgba". Default: "rgb".
            The desired image format.
        target_size: Either `None` (default to original size)
            or tuple of ints `(img_height, img_width)`.
        interpolation: Interpolation and crop methods used to resample and crop the image
            if the target size is different from that of the loaded image.
            Methods are delimited by ":" where first part is interpolation and second is crop
            e.g. "lanczos:random".
            Supported interpolation methods are "nearest", "bilinear", "bicubic", "lanczos",
            "box", "hamming" By default, "nearest" is used.
            Supported crop methods are "none", "center", "random".
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
        ValueError: if interpolation method is not supported.
    """

    # Decode interpolation string. Allowed Crop methods: none, center, random
    interpolation, crop = interpolation.split(":") if ":" in interpolation else (interpolation, "none")  

    if crop == "none":
        return keras_preprocessing.image.utils.load_img(path, 
                                            grayscale=grayscale, 
                                            color_mode=color_mode, 
                                            target_size=target_size,
                                            interpolation=interpolation)

    # Load original size image using Keras
    img = keras_preprocessing.image.utils.load_img(path, 
                                            grayscale=grayscale, 
                                            color_mode=color_mode, 
                                            target_size=None, 
                                            interpolation=interpolation)

    # Crop fraction of total image
    crop_fraction = 0.875
    target_width = target_size[1]
    target_height = target_size[0]

    if target_size is not None:        
        if img.size != (target_width, target_height):

            if crop not in ["center", "random"]:
                raise ValueError('Invalid crop method {} specified.', crop)

            if interpolation not in keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS:
                raise ValueError(
                    'Invalid interpolation method {} specified. Supported '
                    'methods are {}'.format(interpolation,
                        ", ".join(keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS.keys())))
            
            resample = keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS[interpolation]

            width, height = img.size

            # Resize keeping aspect ratio
            # result shold be no smaller than the targer size, include crop fraction overhead
            target_size_before_crop = (target_width/crop_fraction, target_height/crop_fraction)
            ratio = max(target_size_before_crop[0] / width, target_size_before_crop[1] / height)
            target_size_before_crop_keep_ratio = int(width * ratio), int(height * ratio)
            img = img.resize(target_size_before_crop_keep_ratio, resample=resample)

            width, height = img.size

            if crop == "center":
                left_corner = int(round(width/2)) - int(round(target_width/2))
                top_corner = int(round(height/2)) - int(round(target_height/2))
                return img.crop((left_corner, top_corner, left_corner + target_width, top_corner + target_height))
            elif crop == "random":
                left_shift = random.randint(0, int((width - target_width)))
                down_shift = random.randint(0, int((height - target_height)))
                return img.crop((left_shift, down_shift, target_width + left_shift, target_height + down_shift))

    return img
  
keras_preprocessing.image.iterator.load_img = load_and_crop_img

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
fine_datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    rotation_range=15,
    horizontal_flip=True,
    samplewise_std_normalization = True)
fine_dir = "/content/drive/MyDrive/CS135_final_data/fine"
fine_img = fine_datagen.flow_from_directory(fine_dir,target_size=(128, 128), interpolation = 'lanczos:center')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 10116 images belonging to 25 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
batch_size=32
fine_img.reset()
X_fine, y_fine = next(fine_img)
for i in tqdm(range(int(len(fine_img))-1)): #1st batch is already fetched before the for loop.
  img, label = next(fine_img)
  X_fine = np.append(X_fine, img, axis=0)
  y_fine = np.append(y_fine, label, axis=0)
print(X_fine.shape, y_fine.shape)

100%|██████████| 316/316 [2:21:11<00:00, 26.81s/it]

(10116, 128, 128, 3) (10116, 25)


**Fine-To-Coarse Mapping**

(Ideally, this would be done through spectral clustering as opposed to hard-coding)

In [ ]:
fine2coarse = np.zeros((fine_categories,coarse_categories))
fine2coarse[0,0] = 1
fine2coarse[1,5] = 1
fine2coarse[2,5] = 1
fine2coarse[3,0] = 1
fine2coarse[4,4] = 1
fine2coarse[5,4] = 1
fine2coarse[6,3] = 1
fine2coarse[7,6] = 1
fine2coarse[8,4] = 1
fine2coarse[9,2] = 1
fine2coarse[10,5] = 1
fine2coarse[11,4] = 1
fine2coarse[12,8] = 1
fine2coarse[13,5] = 1
fine2coarse[14,4] = 1
fine2coarse[15,2] = 1
fine2coarse[16,4] = 1
fine2coarse[17,6] = 1
fine2coarse[18,7] = 1
fine2coarse[19,3] = 1
fine2coarse[20,8] = 1
fine2coarse[21,5] = 1
fine2coarse[22,1] = 1
fine2coarse[23,5] = 1
fine2coarse[24,6] = 1

In [ ]:
with np.printoptions(threshold=np.inf):
    print(fine2coarse)

**Split Training set into Training and Validation sets**

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_fine, y_fine, test_size=.2,shuffle=True)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(8092, 128, 128, 3)
(8092, 25)


In [ ]:
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

for i in range(0,10):
    image = x_train[i]
    plt.imshow(image)
    plt.show()
    print(np.where(y_train[i] == 1)[0])


for i in range(0,10):
    image = x_val[i]
    plt.imshow(image)
    plt.show()
    print(np.where(y_train[i] == 1)[0])

In [ ]:
y_fine_eight = y_fine[np.argwhere(y_fine==1)[:,1] < 8]
X_fine_eight = X_fine[np.argwhere(y_fine==1)[:,1] < 8]
x_train_eight, x_val_eight, y_train_eight, y_val_eight = train_test_split(X_fine_eight, y_fine_eight, test_size=.1,shuffle=True)

**Constructing CNN**

In [ ]:
from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.models import Model

in_layer = Input(shape=(128, 128, 3), dtype='float32', name='main_input')

net = Conv2D(64, 3, strides=1, padding='same', activation='relu')(in_layer)
net = BatchNormalization()(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(128, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.3)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(256, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.4)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(512, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Flatten()(net)

net = Dense(1024, activation='relu')(net)
net = BatchNormalization()(net)
net = Dropout(.3)(net)
net = Dense(25, activation='softmax')(net)

In [ ]:
import shutil
import gc
dir_path = '/content/data/models'
dir_path1 = '/content/data/graph'

try:
    shutil.rmtree(dir_path)

except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

try:
    shutil.rmtree(dir_path1)

except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

model = 0
net = 0
gc.collect()

Error: /content/data/models : No such file or directory
Error: /content/data/models : No such file or directory


0

**Compile Model**

In [ ]:
model = Model(inputs=in_layer,outputs=net)
adam_coarse =  tf.keras.optimizers.Adam(learning_rate=0.001,epsilon=1e-05)
model.compile(optimizer= adam_coarse, loss='categorical_crossentropy', metrics=['accuracy'])
tbCallBack = kr.callbacks.TensorBoard(log_dir='./data/graph/relu_drop/', histogram_freq=0, write_graph=True, write_images=True)

**Train Model**

In [ ]:
index= 0
step = 5
stop = 30
batch = 64

while index < stop:
    model.fit(x_train, y_train, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val), callbacks=[tbCallBack])
    index += step
    model.save_weights('data/models/model_coarse'+str(index))
save_index = index

Train on 8092 samples, validate on 2024 samples
Epoch 1/5
8092/8092 [==============================] - ETA: 0s - loss: 3.2329 - acc: 0.1789

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


8092/8092 [==============================] - 17s 2ms/sample - loss: 3.2329 - acc: 0.1789 - val_loss: 8.7352 - val_acc: 0.0588
Epoch 2/5
8092/8092 [==============================] - 16s 2ms/sample - loss: 2.5190 - acc: 0.3040 - val_loss: 5.1118 - val_acc: 0.1102
Epoch 3/5
8092/8092 [==============================] - 16s 2ms/sample - loss: 1.9934 - acc: 0.4182 - val_loss: 2.7745 - val_acc: 0.2609
Epoch 4/5
8092/8092 [==============================] - 16s 2ms/sample - loss: 1.5053 - acc: 0.5445 - val_loss: 2.7144 - val_acc: 0.3039
Epoch 5/5
8092/8092 [==============================] - 16s 2ms/sample - loss: 0.9262 - acc: 0.7132 - val_loss: 2.5657 - val_acc: 0.3399
Train on 8092 samples, validate on 2024 samples
Epoch 6/10
8092/8092 [==============================] - 16s 2ms/sample - loss: 0.4415 - acc: 0.8685 - val_loss: 2.5460 - val_acc: 0.3725
Epoch 7/10
8092/8092 [==============================] - 16s 2ms/sample - loss: 0.1819 - acc: 0.9544 - val_loss: 2.8566 - val_acc: 0.3231
Epoch 8/

In [ ]:
index= 0
step = 5
stop = 40
batch = 64
while index < stop:
    model.fit(x_train_eight, y_train_eight, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val_eight, y_val_eight), callbacks=[tbCallBack])
    index += step
    model.save_weights('data/models/model_coarse'+str(index))
save_index = index

Train on 3128 samples, validate on 348 samples
Epoch 1/5
3128/3128 [==============================] - ETA: 0s - loss: 2.3780 - acc: 0.4060

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


3128/3128 [==============================] - 8s 3ms/sample - loss: 2.3780 - acc: 0.4060 - val_loss: 14.4857 - val_acc: 0.1293
Epoch 2/5
3128/3128 [==============================] - 6s 2ms/sample - loss: 1.4230 - acc: 0.5521 - val_loss: 28.9500 - val_acc: 0.1121
Epoch 3/5
3128/3128 [==============================] - 6s 2ms/sample - loss: 0.9822 - acc: 0.6723 - val_loss: 27.6614 - val_acc: 0.1121
Epoch 4/5
3128/3128 [==============================] - 6s 2ms/sample - loss: 0.7144 - acc: 0.7458 - val_loss: 29.5934 - val_acc: 0.1121
Epoch 5/5
3128/3128 [==============================] - 6s 2ms/sample - loss: 0.4873 - acc: 0.8274 - val_loss: 15.3162 - val_acc: 0.1264
Train on 3128 samples, validate on 348 samples
Epoch 6/10
3128/3128 [==============================] - 6s 2ms/sample - loss: 0.3274 - acc: 0.8942 - val_loss: 8.4820 - val_acc: 0.1724
Epoch 7/10
3128/3128 [==============================] - 6s 2ms/sample - loss: 0.2312 - acc: 0.9255 - val_loss: 3.0200 - val_acc: 0.3477
Epoch 8/10


### Load Most Recent Model

In [ ]:
adam_fine =  tf.keras.optimizers.Adam(learning_rate=0.0001,epsilon=1e-05)

In [ ]:
for i in range(len(model.layers)):
    model.layers[i].trainable=False

## Fine-Tuning for Coarse Classifier

In [ ]:
y_train_c = np.dot(y_train,fine2coarse)
y_val_c = np.dot(y_val,fine2coarse)

In [ ]:
print(y_train_c.shape)
print(y_train.shape)
print(fine2coarse.shape)

(8092, 9)
(8092, 25)
(25, 9)


In [ ]:
net = Conv2D(512, 3, strides=1, padding='same', activation='relu')(model.layers[-9].output)
net = BatchNormalization()(net)
# net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Flatten()(net)

net = Dense(1024, activation='relu')(net)
net = BatchNormalization()(net)
net = Dropout(.3)(net)
out_coarse = Dense(9, activation='softmax')(net)

model_c = Model(inputs=in_layer,outputs=out_coarse)
model_c.compile(optimizer= adam_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

In [ ]:
index = 30
step = 10
stop = 40

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 8092 samples, validate on 2024 samples
Epoch 31/40
8092/8092 [==============================] - ETA: 0s - loss: 1.7772 - acc: 0.4668

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


8092/8092 [==============================] - 7s 910us/sample - loss: 1.7772 - acc: 0.4668 - val_loss: 2.2444 - val_acc: 0.4111
Epoch 32/40
8092/8092 [==============================] - 7s 829us/sample - loss: 0.8899 - acc: 0.6901 - val_loss: 1.3489 - val_acc: 0.5677
Epoch 33/40
8092/8092 [==============================] - 7s 832us/sample - loss: 0.4296 - acc: 0.8548 - val_loss: 1.6258 - val_acc: 0.5311
Epoch 34/40
8092/8092 [==============================] - 7s 834us/sample - loss: 0.1573 - acc: 0.9528 - val_loss: 1.6897 - val_acc: 0.5608
Epoch 35/40
8092/8092 [==============================] - 7s 840us/sample - loss: 0.0582 - acc: 0.9867 - val_loss: 1.6680 - val_acc: 0.5707
Epoch 36/40
8092/8092 [==============================] - 7s 839us/sample - loss: 0.0262 - acc: 0.9951 - val_loss: 1.7675 - val_acc: 0.5741
Epoch 37/40
8092/8092 [==============================] - 7s 838us/sample - loss: 0.0133 - acc: 0.9986 - val_loss: 1.8844 - val_acc: 0.5627
Epoch 38/40
8092/8092 [================

In [ ]:
model_c.compile(optimizer= adam_fine, loss='categorical_crossentropy', metrics=['accuracy'])
stop = 50

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 8092 samples, validate on 2024 samples
Epoch 41/50
8092/8092 [==============================] - ETA: 0s - loss: 0.0034 - acc: 0.9998

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


8092/8092 [==============================] - 7s 912us/sample - loss: 0.0034 - acc: 0.9998 - val_loss: 2.4332 - val_acc: 0.5425
Epoch 42/50
8092/8092 [==============================] - 7s 829us/sample - loss: 0.0327 - acc: 0.9906 - val_loss: 2.7144 - val_acc: 0.5469
Epoch 43/50
8092/8092 [==============================] - 7s 834us/sample - loss: 0.0763 - acc: 0.9740 - val_loss: 2.7416 - val_acc: 0.5247
Epoch 44/50
8092/8092 [==============================] - 7s 835us/sample - loss: 0.0714 - acc: 0.9753 - val_loss: 2.4672 - val_acc: 0.5484
Epoch 45/50
8092/8092 [==============================] - 7s 837us/sample - loss: 0.0305 - acc: 0.9896 - val_loss: 2.5381 - val_acc: 0.5405
Epoch 46/50
8092/8092 [==============================] - 7s 837us/sample - loss: 0.0165 - acc: 0.9944 - val_loss: 2.5567 - val_acc: 0.5672
Epoch 47/50
8092/8092 [==============================] - 7s 839us/sample - loss: 0.0154 - acc: 0.9952 - val_loss: 2.5926 - val_acc: 0.5632
Epoch 48/50
8092/8092 [================

## Fine-Tuning for Fine Classifiers

### Construct Fine Classifiers

In [ ]:
def fine_model():

    net = Conv2D(512, 3, strides=1, padding='same', activation='relu')(model.layers[-9].output)
    net = BatchNormalization()(net)
    net = MaxPooling2D((2, 2), padding='valid')(net)

    net = Flatten()(net)

    net = Dense(1024, activation='relu')(net)
    net = BatchNormalization()(net)
    net = Dropout(.3)(net)
    out_fine = Dense(25, activation='softmax')(net)


    model_fine = Model(inputs=in_layer,outputs=out_fine)
    model_fine.compile(optimizer= adam_coarse,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [ ]:
fine_models = {'models' : [{} for i in range(coarse_categories)], 'yhf' : [{} for i in range(coarse_categories)]}
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i

### Train Fine Classifiers on Respective Data

In [ ]:
def get_error(y,yh):
    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [ ]:
for i in range(coarse_categories):
    index= 0
    step = 5
    stop = 5
    
    # Get all training data for the coarse category
    ix = np.where([(y_train[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_tix = x_train[ix]
    y_tix = y_train[ix]
    
    # Get all validation data for the coarse category
    ix_v = np.where([(y_val[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_vix = x_val[ix_v]
    y_vix = y_val[ix_v]
    
    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=16, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
    
    fine_models['models'][i].compile(optimizer=adam_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    stop = 10

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
        
    yh_f = fine_models['models'][i].predict(x_val[ix_v], batch_size=batch)
    print('Fine Classifier '+str(i)+' Error: '+str(get_error(y_val[ix_v],yh_f))) 

Train on 637 samples, validate on 161 samples
Epoch 1/5
637/637 [==============================] - 2s 3ms/sample - loss: 0.0394 - acc: 0.9906 - val_loss: 1.7424 - val_acc: 0.7888
Epoch 2/5
637/637 [==============================] - 1s 1ms/sample - loss: 0.0469 - acc: 0.9874 - val_loss: 1.5489 - val_acc: 0.8012
Epoch 3/5
637/637 [==============================] - 1s 1ms/sample - loss: 0.0221 - acc: 0.9953 - val_loss: 1.3969 - val_acc: 0.8012
Epoch 4/5
637/637 [==============================] - 1s 1ms/sample - loss: 0.0393 - acc: 0.9859 - val_loss: 1.1090 - val_acc: 0.8261
Epoch 5/5
637/637 [==============================] - 1s 1ms/sample - loss: 0.0105 - acc: 0.9969 - val_loss: 1.0110 - val_acc: 0.8199
Train on 637 samples, validate on 161 samples
Epoch 6/10


ResourceExhaustedError: ignored

## Probabilistic Averaging

In [ ]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))
    
    yh_s = model.predict(X, batch_size=batch)
    
    print('Single Classifier Error: '+str(get_error(y,yh_s)))
    
    yh_c = model_c.predict(X, batch_size=batch)
    y_c = np.dot(y,fine2coarse)
    
    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(coarse_categories):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])
    
    print('Overall Error: '+str(get_error(y,yh)))
    return yh

In [ ]:
yh = eval_hdcnn(x_val,y_val)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Single Classifier Error: 0.6556
Coarse Classifier Error: 0.406
Evaluating Fine Classifier:  0
Evaluating Fine Classifier:  5
Evaluating Fine Classifier:  10
Evaluating Fine Classifier:  15
Overall Error: 0.5764
